In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R125_bmimPF6_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 0.3
    init_kappa_1_2A = -0.005
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimPF6'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimPF6','bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'bmimPF6'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data['x_bmimPF6']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R125IL.csv')
print(data)

    temperature  pressure  x_R125  x_bmimPF6
0         283.1      9900   0.013      0.987
1         283.2     99600   0.034      0.966
2         283.3    199600   0.075      0.925
3         283.2    299600   0.126      0.874
4         283.3    399600   0.185      0.815
5         283.1    499700   0.255      0.745
6         283.2    599700   0.342      0.658
7         283.1    699600   0.462      0.538
8         298.2     99600   0.023      0.977
9         298.2    199800   0.050      0.950
10        298.1    299500   0.080      0.920
11        298.2    399800   0.111      0.889
12        298.2    499500   0.143      0.857
13        298.2    599500   0.180      0.820
14        298.2    799600   0.261      0.739
15        298.1    999800   0.363      0.637
16        323.1     99600   0.013      0.987
17        323.1    199800   0.028      0.972
18        323.2    300000   0.042      0.958
19        323.1    399700   0.057      0.943
20        323.2    499900   0.072      0.928
21        

In [4]:
N = 10 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.07 #mole fraction

for i in range(N):
    print(i)
#     noise_T = np.random.normal(loc=0,scale=st_dev_T)
#     print(noise_T)
#     noise_P = np.random.normal(loc=0,scale=st_dev_P)
#     print(noise_P)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
#     data_["temperature"] += noise_T
    data_["x_bmimPF6"] += data_["x_bmimPF6"] * noise_x
#     data_["pressure"] += noise_P
    
#     data_.loc[data_['pressure'] <= 0, 'pressure'] = 1000
    data_.loc[data_['x_bmimPF6'] <= 0.0001, 'x_bmimPF6'] = 0.0001
    data_.loc[data_['x_bmimPF6'] >= 0.9999 , 'x_bmimPF6'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=True)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.10301939618294755
2021-10-28 12:58:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:14 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 12:58:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 12:58:26 [INFO] idaes.init.fs.state_block: Property package initializati

  21  5.1333312e+03 7.07e-01 2.87e+07  -1.0 8.44e+04    -  3.62e-02 3.83e-03h  1
  22  5.1334268e+03 7.06e-01 2.87e+07  -1.0 8.60e+04    -  3.47e-02 3.62e-05h  1
  23  5.1343226e+03 7.06e-01 2.87e+07  -1.0 8.40e+04    -  8.75e-05 3.47e-04h  1
  24  5.3422439e+03 6.51e-01 7.34e+07  -1.0 8.40e+04    -  4.34e-04 7.95e-02h  1
  25  5.3444453e+03 6.50e-01 7.33e+07  -1.0 7.80e+04    -  1.23e-02 8.97e-04h  1
  26  5.3448455e+03 6.50e-01 7.33e+07  -1.0 7.64e+04    -  1.44e-05 1.67e-04h  1
  27  5.3851120e+03 6.39e-01 6.49e+07  -1.0 7.63e+04    -  1.61e-03 1.68e-02h  1
  28  5.3933176e+03 6.37e-01 6.45e+07  -1.0 7.36e+04    -  3.41e-05 3.54e-03h  1
  29r 5.3933176e+03 6.37e-01 1.00e+03  -0.9 0.00e+00    -  0.00e+00 2.53e-07R  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30r 5.3932286e+03 6.20e-01 9.98e+02  -0.9 2.01e+04    -  3.42e-03 1.44e-03f  1
  31r 5.3929956e+03 5.33e-01 9.92e+02  -0.9 1.81e+04    -  1.50e-02 5.39e-03f  1
  32r 5.3930606e+03 4.15e-01

2021-10-28 13:00:37 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:00:39 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:00:42 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Syste

2021-10-28 13:01:41 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:01:43 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:01:45 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:01:46 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:01:49 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:01:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:01:53 [INFO] idaes.i

fs.properties.PR_kappa_A[bmimPF6,R125] = 0.19232409245873727
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.0006493585542307322
3
0.03471630934610191
2021-10-28 13:02:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:02:24 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:02:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:02:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:02:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:02:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:02:33 [INFO] idaes.init.fs.state_block: Property package initiali

...

   4  8.5351558e+02 5.05e+00 1.43e+06  -1.0 2.33e+06    -  1.30e-03 2.40e-03h  6
   5  8.5571195e+02 5.05e+00 1.43e+06  -1.0 1.09e+07    -  2.66e-03 4.33e-05h  2
   6  8.5940485e+02 5.05e+00 1.43e+06  -1.0 6.02e+06    -  1.27e-02 1.33e-04h  1
   7  8.8200703e+02 5.05e+00 1.43e+06  -1.0 3.11e+06    -  1.29e-01 1.23e-03h  5
   8  9.1782809e+02 5.04e+00 1.43e+06  -1.0 2.56e+06    -  2.22e-01 1.77e-03h  8
   9  8.9931576e+02 4.27e+00 9.57e+06  -1.0 2.05e+05  -4.0 8.90e-05 1.56e-01F  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  9.0454614e+02 4.27e+00 9.56e+06  -1.0 5.93e+05    -  2.88e-02 8.28e-04h  1
  11  9.0466490e+02 4.27e+00 9.56e+06  -1.0 5.93e+05    -  2.34e-02 2.84e-05h  1
  12  9.3502202e+02 4.23e+00 9.47e+06  -1.0 5.93e+05    -  7.70e-02 9.11e-03h  6
  13  1.0032286e+03 4.15e+00 9.29e+06  -1.0 5.93e+05    -  1.27e-01 1.84e-02h  5
  14  1.0233229e+03 4.13e+00 9.24e+06  -1.0 6.35e+05    -  2.75e-01 4.74e-03h  7
  15  1.0344857e+03 4.13e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  4.5755650e+05 3.02e-01 5.19e+08  -1.0 2.47e+04  -3.3 6.46e-01 9.90e-01f  1
 101  4.5693851e+05 1.16e+00 3.28e+07  -1.0 2.49e+04    -  9.58e-01 9.91e-01f  1
 102  4.5665811e+05 3.37e-01 1.12e+07  -1.0 2.05e+05    -  7.95e-01 7.09e-01f  1
 103  4.5689055e+05 3.41e-02 7.01e+06  -1.0 4.36e+04    -  6.37e-02 1.00e+00h  1
 104  4.5688345e+05 2.59e-03 2.06e+06  -1.0 2.02e+04    -  7.75e-01 1.00e+00f  1
 105  4.5689387e+05 4.34e-05 5.88e+04  -1.0 2.52e+03    -  9.90e-01 1.00e+00h  1
 106  4.5689408e+05 6.26e-08 2.00e+02  -1.0 1.08e+02    -  9.90e-01 1.00e+00h  1
 107  4.5689398e+05 2.26e-08 2.06e+04  -2.5 6.31e+01    -  9.98e-01 1.00e+00f  1
 108  4.5689398e+05 7.05e-12 2.59e-06  -2.5 8.48e-02    -  1.00e+00 1.00e+00h  1
 109  4.5689398e+05 1.86e-11 2.69e+01  -8.6 1.84e+00    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  4.5689398e+05 4.89e-12

   4  8.3290085e+02 3.36e+00 1.20e+06  -1.0 5.05e+04    -  1.99e-03 4.55e-02h  2
   5  8.3197792e+02 3.34e+00 2.36e+06  -1.0 1.78e+05    -  5.85e-03 8.55e-03F  1
   6  8.6265975e+02 2.94e+00 2.01e+06  -1.0 5.85e+04    -  4.09e-02 1.30e-01h  1
   7  8.6421207e+02 2.92e+00 2.00e+06  -1.0 4.86e+04    -  6.91e-03 6.85e-03h  7
   8  8.0406683e+02 2.89e+00 1.88e+06  -1.0 5.75e+04    -  5.51e-02 4.85e-02H  1
   9  8.1945270e+02 2.75e+00 1.79e+06  -1.0 6.90e+04    -  7.25e-01 4.94e-02h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  8.2087468e+02 2.72e+00 1.78e+06  -1.0 7.87e+04    -  4.94e-01 9.05e-03h  2
  11  8.2091106e+02 2.72e+00 1.78e+06  -1.0 6.86e+04    -  4.56e-01 3.37e-04h  7
  12  8.6218005e+02 2.12e+00 1.17e+06  -1.0 6.62e+04    -  2.36e-01 2.36e-01s 20
  13  8.6229336e+02 2.12e+00 1.17e+06  -1.0 4.59e+04    -  5.64e-04 5.64e-04s 20
  14r 8.6229336e+02 2.12e+00 1.00e+03  -0.0 0.00e+00    -  0.00e+00 0.00e+00R  1
  15r 8.6262322e+02 1.97e+00

2021-10-28 13:11:57 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:11:58 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:11:59 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:12:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:12:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:12:03 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:12:04 [INFO] idaes.init.fs.state_block: Property package initializat

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  8.3552807e+02 1.44e+00 3.04e+04  -1.7 7.05e+04    -  1.00e+00 1.00e+00f  1
  51  1.2408052e+03 1.03e+00 2.78e+04  -1.7 9.11e+04    -  1.00e+00 6.00e-01h  1
  52  8.0815017e+02 1.69e+00 2.59e+04  -1.7 7.03e+04    -  1.00e+00 1.00e+00f  1
  53  1.0746693e+03 8.44e-01 2.74e+04  -1.7 1.06e+05    -  1.00e+00 5.00e-01h  2
  54  1.7970423e+03 5.50e-03 2.10e+04  -1.7 7.25e+04    -  1.00e+00 1.00e+00h  1
  55  1.8064614e+03 1.67e-05 5.32e+01  -1.7 3.24e+03    -  1.00e+00 1.00e+00h  1
  56  1.8064928e+03 2.12e-07 3.20e-02  -2.5 1.56e+01    -  1.00e+00 1.00e+00h  1
  57  1.8064902e+03 7.10e-11 7.04e-04  -5.7 1.60e-01    -  1.00e+00 1.00e+00f  1
  58  1.8064902e+03 3.92e-12 2.86e-09  -5.7 6.02e-05    -  1.00e+00 1.00e+00h  1
  59  1.8064902e+03 3.92e-12 1.06e-09  -8.6 9.75e-05    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 59

                                   (scaled)                 (unscaled)
Objecti

In [5]:
print(results_params)

[[ 0.5155547   0.00990573]
 [ 0.29602318  0.00139565]
 [ 0.19232409  0.00064936]
 [ 0.35166581  0.00269887]
 [ 0.17455906 -0.00176016]
 [ 0.26752745  0.00084616]
 [ 0.13434864 -0.00129399]
 [ 0.10059115 -0.00355916]
 [ 0.35702118  0.0037179 ]
 [ 0.3093167   0.00178243]]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_bmim_07.csv')